## Step 1

Load in the data using python requests. The last line of the url, 'order=date+desc', tells us to order the data by the date with the most recent data being called in first.

In [60]:
import os
import requests
import json

response_API = requests.get('https://healthdata.gov/resource/j8mb-icvb.json?$order=date+desc')
# response_API = requests.get('https://healthdata.gov/resource/j8mb-icvb.json?$limit=50000&$offset=124375')

data = response_API.text
data = json.loads(data)

## Step 2

Create a list of all the states in the data which we will use later on.

In [61]:
unique_states = [d['state_name'] for d in data]
unique_states = set(unique_states)

## Step 3

Create a list of the last 5 days for which we have data.

In [62]:
from datetime import datetime  
from datetime import timedelta

latest_data = max(data, key=lambda d: d['date'])
days = [str(datetime.strptime(latest_data['date'][ : 10], '%Y-%m-%d')-timedelta(days=i)) for i in range(5)]

## Step 4

Iterate through each state, creating a new word document for each state we encounter. The heading will be the state name followed by today's date. We create a table with the 5 columns requested in the assessment documentation. We then iterate through each day and row of data and when the state and day match the current iteration values, and the outcome is positive, we add to the columns for positive tests with the corresponding values for that row of data. For example, if we are on Alabama and the iteration date is 2022-04-10, if the row of data in the current iteration (represented by 'd' in this code) has state_name Alabama, date 2022-04-10 and the overall_outcome positive, we add the values of new_results_reported to the 'New Positive Tests Reported'column in our table, and we add the values of total_results_reported to the 'Total Positive Tests Reported' column. The same process is followed for the negative results. The results are then saved in a specified filepath location.

In [ ]:
import docx
from datetime import date
from zipfile import ZipFile

filepath = os.path.normpath('YourFilepath')
zip_ = ZipFile(filepath + 'HexegonAssessment.zip', 'w')

for state in unique_states:
    doc = docx.Document()
    doc.add_heading(state + ' ' + '-' + ' ' + str(date.today()), 0)
    table = doc.add_table(rows=1, cols=5)
    row = table.rows[0].cells
    row[0].text = 'Date'
    row[1].text = 'New Positive Tests Reported'
    row[2].text = 'New Negative Tests Reported'
    row[3].text = 'Total Positive Tests Reported'
    row[4].text = 'Total Negative Tests Reported'
    
    for day in days:
        for d in data:
            if str(datetime.strptime(d['date'][ : 10], '%Y-%m-%d'))==day and d['state_name']==state and d['overall_outcome']=='Positive':
                row = table.add_row().cells
                row[0].text = str(datetime.strptime(d['date'][ : 10], '%Y-%m-%d'))
                row[1].text = str(d['new_results_reported'])
                row[2].text = str(d['total_results_reported'])
            elif str(datetime.strptime(d['date'][ : 10], '%Y-%m-%d'))==day and d['state_name']==state and d['overall_outcome']=='Negative':
                row = table.add_row().cells
                row[3].text = str(d['new_results_reported'])
                row[4].text = str(d['total_results_reported'])
      
    doc.save(filepath + '\\' + state + '_test_results.docx')
    zip_.write(filepath + '\\' + state + '_test_results.docx')
    
os.remove(filepath)
zip_.close()